# 使用 sqlite-vss

未完成。

## sqlite-vss 简介

https://github.com/asg017/sqlite-vss

sqlite-vss（SQLite Vector Similarity Search ）是一个 SQLite扩展，它基于Faiss为 SQLite 带来了向量搜索功能。它可用于构建语义搜索引擎、推荐或问答工具。

## 安装

In [1]:
!apt-get update
!apt-get install -y libgomp1 libatlas-base-dev liblapack-dev

Get:1 https://developer.download.nvidia.cn/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 https://developer.download.nvidia.cn/compute/cuda/repos/ubuntu2204/x86_64  Packages [814 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]      
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]      
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]      
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2259 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1369 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2339 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2035 kB]
Get:12 http://archive.ubuntu.com/ubuntu ja

In [2]:
!pip install sqlite-vss

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00a 0:00:01


## 测试可用性

In [1]:
# 加载内存数据库和插件

import sqlite3
import sqlite_vss

db = sqlite3.connect(':memory:')
db.enable_load_extension(True)
sqlite_vss.load(db)
db.enable_load_extension(False)

version, = db.execute('select vss_version()').fetchone()
print(version)

v0.1.2


In [2]:
# 创建存储表

# 创建一个游标对象
cursor = db.cursor()

# 创建一个名为 example_table 的表，包含一个字段 name（文本类型）
cursor.execute('''
create virtual table vss_demo using vss0(
  a(2)
)
''')

# 提交更改
db.commit()

In [3]:
# 插入一个向量字段

import json

embedding = [0.1, 0.2, 0.3]
db.execute("insert into vss_demo(a) values (?)", [json.dumps(embedding)])

In [4]:
# 建虚拟表
cursor.execute('''
create virtual table vss_xyz using vss0(
  headline_embedding(1024),
  description_embedding(1024) 
)
''')

In [5]:
# 训练
cursor.execute('''
insert into vss_xyz(operation, description_embedding)
  select 'training', description_embedding from vss_xyz
''')

In [6]:
# 插入数据
cursor.execute('''
insert into vss_xyz(rowid, headline_embedding, description_embedding)
  select rowid, headline_embedding, description_embedding from vss_xyz
''')

In [7]:
cursor.execute('''
insert into vss_xyz(headline_embedding)
  values ('[0.1, 0.2, 0.3]')
''')

In [8]:
# 提交更改
db.commit()

In [9]:
cursor.execute('''
select * from vss_xyz
''')

OperationalError: Error reconstructing vector - Does the column factory string end with IDMap2? Full error: Error in void faiss::IndexIDMap2Template<IndexT>::reconstruct(faiss::idx_t, typename IndexT::component_t*) const [with IndexT = faiss::Index; faiss::idx_t = long int; typename IndexT::component_t = float] at /home/runner/work/sqlite-vss/sqlite-vss/vendor/faiss/faiss/IndexIDMap.cpp:236: key 0 not found